<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 7.2**
---

# **BLIP**

BLIP (Bootstrapping Language-Image Pre-training) es un modelo **multimodal** que combina Visión por Computador (CV) y PLN ([Junnan et al., 2022](https://arxiv.org/abs/2201.12086)). Está pensado para trabajar con pares imagen-texto y se usa para tareas como:

* **Image captioning** (describir imágenes con texto).
* **Visual Question Answering (VQA)** (responder preguntas sobre una imagen).
* **Cross-modal retrieval** (buscar imágenes a partir de texto y viceversa).
* En general, cualquier tarea donde haya que **entender conjuntamente imagen y lenguaje**.


## **Arquitectura general de BLIP**

BLIP tiene una arquitectura **encoder–decoder multimodal**, pensada para tanto **comprender** como **generar** texto condicionado por imágenes. 

<div align="center">
<img src="./imgs/blip.png" alt="BLIP Architecture" width="900"/>
</div>


La estructura básica tiene tres componentes principales:

### **Unimodal Encoder (Codificador unimodal)**

Se encarga de codificar **por separado** imagen y texto:

* La imagen se codifica típicamente con un backbone tipo ViT / CNN que ofrece como resultados *embeddings* visuales.
* El texto se procesa con un transformer de lenguaje que produce *embeddings* de tokens.

En esta parte aún no hay interacción fuerte entre modalidades; se generan representaciones “puras” de cada una.

### **Image-grounded Text Encoder (Codificador de texto condicionado por imagen)**

Aquí ya hablamos de un transformer de texto que **incorpora información visual** mediante **cross-attention**. El flujo de procesamiento es:

* Recibe como entrada los tokens de texto (p.ej. una frase candidata o un prompt).
* Tiene acceso a los embeddings de la imagen como “memoria” en las capas de atención cruzada.
* El objetivo es producir una representación de texto que **esté alineada con la imagen**; es decir, el texto se codifica sabiendo qué se ve en la imagen.

### **Image-grounded Text Decoder (Decodificador de texto condicionado por imagen)**
Es la parte **generativa** del modelo:

* Usa **self-attention causal** (como un GPT) en los tokens de salida.
* Además, usa **cross-attention** para consultar la representación visual.
* A partir de la imagen (y opcionalmente un prompt textual), el decodificador va generando la secuencia palabra a palabra.

En conjunto:

* El encoder unimodal y el encoder “image-grounded” sirven para **entender** (clasificación, matching, VQA…).
* El decoder “image-grounded” sirve para **generar texto** (captioning, respuestas completas, etc.).


## **Objetivos de preentrenamiento de BLIP**

El entrenamiento de BLIP se basa en **tres objetivos principales**, combinando contrastive learning, matching y language modeling.

### **Image-Text Contrastive Loss (ITC)**

El **ITC** es un objetivo de entrenamiento inspirado en CLIP que obliga al modelo a aprender un **espacio latente común** donde:

* la **imagen** y su **texto correcto** estén **muy cerca**,
* la **imagen** y textos **incorrectos** estén **muy lejos**,

Su función principal es **alinear el encoder de imagen y el encoder de texto** antes de combinar modalidades más complejas.En BLIP, este loss se aplica sobre el **vision encoder** (ViT normalmente), y el **unimodal text encoder** (transformer textual sin intervención visual). Esto es importante porque si no existiera este alineamiento previo, no existiría una relación numérica clara entre "lo que ve" y "lo que lee". Una vez alineados ambos espacios latentes, el resto de componentes del modelo pueden saber que "dog", "black dog", "a dog running" y "animal" están cerca de la imagen de un perro, mientras que "car" o "computer" deben estar lejos.

Para el entrenamiento con ITC debemos contar con un conjunto de datos de pares (imagen, texto). El loss se calcula así:

$$
L_{ITC} = - \frac{1}{N} \sum_{i=1}^{N} \left( \log \frac{\exp(\text{sim}(v_i, t_i) / \tau)}{\sum_{j=1}^{N} \exp(\text{sim}(v_i, t_j) / \tau)} + \log \frac{\exp(\text{sim}(t_i, v_i) / \tau)}{\sum_{j=1}^{N} \exp(\text{sim}(t_i, v_j) / \tau)} \right)
$$

* $N$: tamaño del batch (= número de pares imagen–texto).
* $v_i$: embedding **visual** de la imagen $i$.
* $t_i$: embedding **textual** del texto $i$.
* $\text{sim}(v_i, t_j)$: **similitud coseno** entre la imagen $i$ y el texto $j$.
* $\tau$: *temperatura*, un hiperparámetro que controla la "dureza" de la distribución softmax.


Esta fórmula puede parecer compleja, pero vamos a desglosarla paso a paso. El primer término dentro del sumatorio es:

$$
\log \frac{\exp(\text{sim}(v_i, t_i) / \tau)}{\sum_{j=1}^{N} \exp(\text{sim}(v_i, t_j) / \tau)}
$$  

Si te fijas, es una **probabilidad softmax** que mide qué tan probable es que el texto correcto $t_i$ sea el más similar a la imagen $v_i$ entre todos los textos del batch. Por tanto, este término fuerza al modelo a que la similitud entre **imagen $i$** y su **texto correcto $i$** sea **alta**, y que la similitud entre **imagen $i$** y textos **incorrectos $j\neq i$** sea **baja**. Y esto lo hace tanto en la dirección **imagen → texto** como en la dirección **texto → imagen** (segundo sumando de la expresión de la pérdida). Es decir, busca alineamiento **bidireccional**.

La división por $N$ promedia la pérdida sobre todos los pares del batch, y el signo negativo convierte la maximización de la probabilidad en minimización de la pérdida. En cuanto al hiperparámetro $\tau$, controla la "dureza" de la distribución softmax: valores bajos hacen que el modelo se enfoque más en los ejemplos difíciles.


### **Image-Text Matching Loss (ITM)**

Aquí se plantea un problema de **clasificación binaria**. Dado un par (imagen, texto), el modelo debe decidir si el texto **realmente describe** la imagen o no. Para ello se usa la representación multimodal (texto condicionado por imagen). El modelo **predice “match / no match”** forzando al modelo a aprender **representaciones más finas** que las del puro contraste global: no solo "parecidos", sino "¿esta frase encaja exactamente con esta imagen?".

Su expresión matemática es la de una pérdida de entropía cruzada binaria estándar:

$$
L_{ITM} = - \frac{1}{M} \sum_{i=1}^{M} \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
$$

* $M$: número total de pares (imagen, texto) usados en este loss (incluye positivos y negativos).
* $y_i \in \{0,1\}$: etiqueta binaria que indica si el par $i$ es positivo (1) o negativo (0).
* $p_i$: probabilidad predicha por el modelo de que el par $i$ sea positivo (match).


La pérdida ITM ocupa un papel fundamental porque permite que el modelo aprenda a distinguir cuándo una imagen y un texto realmente están relacionados y cuándo no. A diferencia de la pérdida contrastiva ITC (que trabaja de forma global con similitudes en un espacio vectorial común) la ITM opera a un nivel más fino, evaluando la correspondencia semántica entre una imagen concreta y una frase específica. Podríamos decir que la ITM enseña al modelo a “leer” la relación imagen–texto con mayor detalle y no solo a nivel de orientación global en el embedding space.

El mecanismo funciona del siguiente modo. BLIP toma una imagen y su descripción correcta (el par positivo) y posteriormente genera versiones negativas, normalmente seleccionando textos que no corresponden a esa imagen o imágenes que no corresponden a ese texto. Con estos pares mezclados, el modelo debe predecir si la combinación es verdadera o falsa. Para ello utiliza un clasificador binario montado sobre el encoder multimodal, el cual recibe tanto los embeddings visuales como los tokens textuales y produce una probabilidad de que ese par encaje semánticamente.

Este clasificador se entrena con una pérdida de tipo binary cross-entropy. Las parejas auténticas deben obtener una probabilidad alta, mientras que las parejas erróneas deben quedar relegadas a valores bajos. Con este proceso, BLIP no solo aprende qué imágenes y textos son globalmente similares —como hace ITC— sino que además desarrolla sensibilidad hacia relaciones más sutiles: detalles objetuales, interacciones entre elementos, atributos concretos o descripciones incorrectas pero plausibles. En otras palabras, la ITM obliga al modelo a comprender contenido, no solo alineación vectorial.


### **Language Modeling Loss (LM)**

El tercer componente funcional de BLIP es su **módulo de modelado del lenguaje**, responsable de que el modelo no solo pueda alinear imagen y texto (ITC) o verificar si coinciden (ITM), sino también **generar descripciones completas, naturales y precisas** a partir de una imagen. Esta capacidad generativa es esencial para tareas como *image captioning* o *visual question answering*, en las que el modelo debe producir una secuencia lingüística condicionada por la información visual.

La arquitectura que BLIP utiliza para esta parte sigue el patrón clásico de los modelos **encoder–decoder**:

* el *encoder multimodal* —que combina los tokens visuales procedentes del ViT y los tokens textuales de entrada— sirve como **contexto**,
* y un **decodificador lingüístico autoregresivo**, basado en un Transformer unidireccional, genera palabra por palabra la descripción final.

Lo importante es que el decodificador no genera texto de manera aislada, sino atendiendo explícitamente a los **embeddings visuales** mediante mecanismos de *cross-attention*. Esto permite que cada palabra generada esté informada por la imagen. Por ejemplo, para generar la palabra "gato", el decodificador no solo se guía por la estructura sintáctica de la frase, sino también por los patrones visuales en los que se detecta un gato.

La pérdida utilizada para entrenar este módulo es la **pérdida de modelado del lenguaje**, también conocida como **pérdida de entropía cruzada autoregresiva**. Formalmente, el decodificador intenta maximizar la probabilidad de la secuencia textual correcta $T = (t_1, t_2, \dots, t_n)$ dadas la imagen $I$ y el contexto anterior:

$$
\mathcal{L}_{LM} = - \sum_{k=1}^{n} \log P(t_k \mid t_{<k}, I).
$$

Este objetivo fuerza al modelo a predecir correctamente cada token de la descripción, condicionándose tanto en los tokens previos como en los elementos visuales. En otras palabras, la pérdida LM enseña al modelo **gramática, semántica y correspondencia visual–lingüística en un espacio generativo**.

BLIP incluye además un mecanismo interesante llamado *captioning bootstrapping*: durante el entrenamiento temprano, el texto asociado a muchas imágenes web es ruidoso o irrelevante. Gracias a sus módulos ITC e ITM, el modelo es capaz de identificar pares de baja calidad y generar sus **propias descripciones limpias** utilizando el decodificador. Estas descripciones generadas, conocidas como *pseudo-captions*, se utilizan después como datos adicionales para mejorar el entrenamiento. Es un proceso de auto-mejora que incrementa notablemente la calidad final del modelo.

En conjunto, la pérdida LM convierte a BLIP en un sistema plenamente multimodal, capaz de **entender una imagen hasta el punto de describirla con lenguaje natural**. Mientras ITC entrena la alineación global imagen–texto, e ITM enseña la verificación fina de correspondencias, la pérdida LM confiere al modelo la capacidad de producir lenguaje de forma fluida, detallada y contextualmente informada. Esta integración de comprensión, alineación y generación es lo que hizo que BLIP representara un avance significativo en los modelos multimodales previos a la llegada de BLIP-2 y los LLM modernos.

## **Ejemplo práctico con Hugging Face Transformers**

El modelo BLIP que vamos a usar tiene dos modos de funcionamiento principales:
* **Captioning**: genera una descripción textual completa a partir de una imagen.
* **Visual Question Answering (VQA)**: responde a preguntas específicas sobre el contenido de una imagen.


In [1]:
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

processor = BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')

image = Image.open("imgs/rugby.png")
inputs = processor(images=image, return_tensors="pt")

output = model.generate(**inputs)

caption = processor.decode(output[0], skip_special_tokens=True)
print("Generated Caption:", caption)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <3EF8D74C-9F11-3C9E-85DB-9E76BCCBE7A0> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <B6BD92AE-4D03-3F92-9E03-2E2594A12866> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch

Generated Caption: a man in a parking holding a white ball


<div align="center">
    <img src="imgs/perro.png" width="230px" align="left">
</div>

In [3]:
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForQuestionAnswering

# Carga el procesador y el modelo de VQA
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

image = Image.open("imgs/rugby.png")

# Tu pregunta
question = "How many cars are there?"

inputs = processor(image, question, return_tensors="pt")
output = model.generate(**inputs)

answer = processor.decode(output[0], skip_special_tokens=True)
print("Respuesta:", answer)

Respuesta: 5


In [26]:
# Tu pregunta
question = "Where is the man?"

inputs = processor(image, question, return_tensors="pt")
output = model.generate(**inputs)

answer = processor.decode(output[0], skip_special_tokens=True)
print("Respuesta:", answer)

Respuesta: in parking lot
